## PDS API demo
# Visualization of Osiris-Rex OVIRS spectrometer observations of asteroid BENNU

The purpose of this notebook is to demostrate how the PDS web API can be used to access the PDS data for a scientific use case.

Documention for the PDS web API is available on https://nasa-pds.github.io/pds-api/

Documentation for the `peppi` library used to interface with the PDS web API is available on https://nasa-pds.github.io/peppi/

This notebook is available on https://github.com/NASA-PDS/pds-api-notebook

 
<b>WARNING:</b> This notebook is a demo and not a real scientific use case. It might contain mistakes in the way the data is used or displayed.

In [ ]:
from __future__ import print_function
from pprint import pprint, pformat
from IPython.lib.pretty import pretty
import time
import matplotlib.pyplot as plt
from astropy.io import fits
import requests
from requests.exceptions import HTTPError

## List requestable properties

Available properties can be identified with a request, for example, searching for products which match free text search on `ovir`.

Properties are defined in:
- core dictionnary (pds: prefix) : https://pds.nasa.gov/datastandards/dictionaries/index-1.20.0.0.shtml#pds-common
- displine dictionnaries (e.g. geom:) : https://pds.nasa.gov/datastandards/dictionaries/index-1.19.0.0.shtml#discipline-dictionaries
- mission dictionnaries (e.g. orex:) : https://pds.nasa.gov/datastandards/dictionaries/index-missions.shtml#mission-dictionaries

In [ ]:
import pds.api_client as pds_api
from pds.api_client.api.all_products_api import AllProductsApi

configuration = pds_api.Configuration()

# production server
configuration.host = 'https://pds.nasa.gov/api/search/1/'
api_client = pds_api.ApiClient(configuration)

products_api = AllProductsApi(api_client)
api_response = products_api.product_list(keywords=['ovirs'])
api_response.summary.properties

## Access the PDS web API server via peppi

Using the `pds.api_client` is sufficient for this type of exploration, however, the PDS API can be more easily accessed using `peppi`, a PDS Python client library available from https://nasa-pds.github.io/peppi/

In [ ]:
import pds.peppi as pep

Once installed, `peppi` can be used to easily interface with the PDS web API via two objects:

* An instance of the `PDSRegistryClient` class, which handles connections to the PDS web API endpoint
* An instance of the `Products` class, which is used to assemble and submit user-defined queries on the PDS API

In [ ]:
# The PDSRegistryClient class requires no arguments, and is configured with appropriate defaults to communicate with 
# the production PDS Registry API.
client = pep.PDSRegistryClient()

# A Products class instance requires a PDSRegistryClient object that is uses to submit user-defined queries.
# Results from a query are obtained by iterating over a Products class instance after a query has been defined.
products = pep.Products(client)

## Use `peppi` to Get observations around specific spot (lat=12, lon=24) with distance to bennu closer than 4 km

Get the result found in part1 by directly posting the search criteria to the API

We use the `has_target` query clause to locate products target LIDVID "urn:nasa:pds:context:target:asteroid.101955_bennu".

We also use the `has_instrument` query clause to further filter resulting products to "urn:nasa:pds:context:instrument:ovirs.orex" instrument.

Further refinement of the query based on value ranges of specific properties is accomplished via multiple calls to `filter`, which accepts an arbitrary query clause using the PDS API query syntax.

The query syntax for the `filter` calls uses the following operators:
- comparison: lt, le, gt, ge ...

See https://github.com/NASA-PDS/pds-api/blob/master/docs/spec/pds-api-specification.md#query-syntax for detailed syntax.

Lastly, we request the FITS data file url field from the API using the `fields` method.

By chaining these calls together, each portion of the query is connected via logical AND.

In [ ]:
start_time = time.time()

properties_of_interest = ['ref_lid_target', 
                          'ref_lid_instrument', 
                          'orex:Spatial.orex:latitude', 
                          'orex:Spatial.orex:longitude', 
                          'orex:Spatial.orex:target_range', 
                          'ops:Data_File_Info.ops:file_ref']

products.has_target("urn:nasa:pds:context:target:asteroid.101955_bennu") \
        .has_instrument("urn:nasa:pds:context:instrument:ovirs.orex")    \
        .filter("orex:Spatial.orex:target_range lt 4.0")                 \
        .filter("orex:Spatial.orex:latitude ge 9.0")                     \
        .filter("orex:Spatial.orex:latitude le 15.0")                    \
        .filter("orex:Spatial.orex:longitude ge 21.0")                   \
        .filter("orex:Spatial.orex:longitude le 27.0")                   \
        .fields(properties_of_interest)

closer_products = [p for p in products]

elapsed = time.time() - start_time

print(f"Found {len(closer_products)} product(s) in {elapsed} seconds")

## Structure of the reponse

Each product returned from the query results is an object of type [pds.api_client.models.pds_product.PDSProduct](https://nasa-pds.github.io/pds-api-client/api/pds.api_client.models.pds_product.html#pds.api_client.models.pds_product.PdsProduct), which can be rendered as a human readable JSON-format string via the `to_str()` function.

In [ ]:
print(closer_products[0].to_str())

### Check that we have the right lidvids

In [ ]:
lidvids = [product.id for product in closer_products]
print(f'The lidvids of the selected products are:\n{pformat(lidvids)}')

### Extract the links to the data files

In [ ]:
data_files = [product.properties['ops:Data_File_Info.ops:file_ref'][0] for product in closer_products]
print(f"Product data file urls:\n{pformat(data_files)}")

## Plot the data (FITS files)

Plot the spectra of the 4 observations on the same figure, one figure per dimension of the instrument.


In [ ]:
hduls = []

# Read in data, skipping any unreachable links
for data_file in data_files:
    try:
        requests.get(data_file).raise_for_status()
    except HTTPError as err:
        if err.response.status_code == 404:
            # Some URLs may be dead links, in which case just skip them
            continue
        else:
            # Something else went wrong
            raise
            
    hduls.append(fits.open(data_file))

for i in range(20): # for each dimension of the instrument
    fig, ax = plt.subplots()
    ax.set_title(f'detector {i}')
    for hdul in hduls: # for each observation
        ax.plot(hdul[2].data[0, i, :], hdul[0].data[0, i, :])
    ax.set_xlabel('wavelength (micrometers)')
    ax.set_ylabel('radiance')